In [7]:
from people_segmentation.pre_trained_models import create_model
from iglovikov_helper_functions.utils.image_utils import load_rgb, pad, unpad
from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image
from pylab import imshow
import numpy as np
import torch
import cv2
import albumentations as albu

In [8]:
model = create_model("Unet_2020-07-20")

In [9]:
model.eval()

Unet(
  (encoder): EfficientNetEncoder(
    (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): Swish()
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNormAct2d(
            40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Swish()
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): Swish()
            (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False

In [5]:
# define a video capture object 
vid = cv2.VideoCapture(0) 
  
i = 0

old_mask = None

while(True):
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
  
    frame = cv2.resize(frame, (250, 250))

    transform = albu.Compose([albu.Normalize(p=1)], p=1)

    padded_image, pads = pad(frame, factor=32, border=cv2.BORDER_CONSTANT)

    x = transform(image=padded_image)["image"]
    x = torch.unsqueeze(tensor_from_rgb_image(x), 0)

    with torch.no_grad():
        prediction = model(x)[0][0]

    mask = (prediction > 0).cpu().numpy().astype(np.uint8)

    mask = unpad(mask, pads)
    
    mask = mask.astype(bool)

    if not old_mask is None:
        tmp = old_mask
        old_mask = mask
        mask = mask | tmp
    else:
        old_mask = mask

    effect = frame.copy()
    
    effect[:,:,0] = 0
    effect[:,:,2] = 0

    dst = frame.copy()

    dst[mask] = effect[mask]

    dst = cv2.resize(dst, (750, 750))

    # Display the resulting frame 
    cv2.imshow('frame', dst) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

    i += 1
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

In [19]:
# define a video capture object 
vid = cv2.VideoCapture(0) 
  
i = 0

old_mask = None

while(True):
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
  
    frame = cv2.resize(frame, (250, 250))

    transform = albu.Compose([albu.Normalize(p=1)], p=1)

    padded_image, pads = pad(frame, factor=32, border=cv2.BORDER_CONSTANT)

    x = transform(image=padded_image)["image"]
    x = torch.unsqueeze(tensor_from_rgb_image(x), 0)

    with torch.no_grad():
        prediction = model(x)[0][0]

    mask = (prediction > 0).cpu().numpy().astype(np.uint8)

    mask = unpad(mask, pads)
    
    mask = mask.astype(bool)

    if not old_mask is None:
        tmp = old_mask
        old_mask = mask
        mask = mask | tmp
    else:
        old_mask = mask

    effect = cv2.GaussianBlur(frame, (55, 55), 0)

    dst = frame.copy()

    dst[mask] = effect[mask]

    dst = cv2.resize(dst, (750, 750))

    # Display the resulting frame 
    cv2.imshow('frame', dst) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

    i += 1
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 